In [1]:
import io
import subprocess
import os
import glob
import sys
import pwd

import pandas as pd
import numpy as np
import scipy
import scipy.stats as stats
from sklearn.decomposition import PCA
import pickle
import re
import math
import plotnine as pn
import seaborn as sns
import glob
import subprocess
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import seaborn as sns
from numpy import log
from collections import defaultdict,Counter
import re
import statsmodels
import warnings
import statsmodels.stats.multitest as smm
import json

from pathlib import Path
from midas import m
from datetime import datetime, date
import altair as alt

# gh specific
sys.path.append('/ghsfa/projects/pharma/shared_scripts/')
import bibp.functions as functions
import hgvslib.pHGVS as pHGVS  ## do a pull on the hgvslib git repo

# !pip install statannotations==0.4.0
# #!pip uninstall statannotations --yes
# from statannotations.Annotator import Annotator

warnings.filterwarnings("ignore")
pd.options.display.max_rows = None
pd.options.display.max_columns=None
pd.options.display.max_colwidth=None
work_dir="/ghsfa/projects/pharma/projects/sirius_pharma/hazhang_projects/AZ_2024cohorts_prevalence_warning_08282024"

# Pull all AZ batches from pharma data warehouse (Infinity, OMNI and G360)

In [8]:
# File name list to handle corner case of data readin
header1_list = [""]
header0_list = [
 '20230928_AstraZeneca_SOW164_46Samples_InfinityReport.csv',
 '20231106_AstraZeneca_SOW164_46Samples_InfinityReport.csv',
 '20231207_AstraZeneca_SOW170_188Samples_InfinityReport.csv',
 '20240111_AstraZeneca_SOW174_35Samples_InfinityReport.csv',
 '20240116_AstraZeneca_SOW182_40Samples_InfinityReport.csv',
 '20240422_AstraZeneca_SOW176Monitoring_998Samples_InfinityReport.csv',

]

# Define the folder paths
folders = [
    # "/ghsfa/projects/pharma/datawarehouse/Infinity", # This only include AZ batches before 2024
    "/ghsfa/projects/pharma/datawarehouse/infinity",
    "/ghsfa/projects/pharma/datawarehouse/data/infinity_ist_report_warehouse",
    "/ghsfa/projects/pharma/datawarehouse/data/infinity_mrd_report_warehouse",
    "/ghsfa/projects/pharma/datawarehouse/data/infinity_report_warehouse",

    "/ghsfa/projects/pharma/datawarehouse/data/g360_reports_since_201902", # For G360
    "/ghsfa/projects/pharma/datawarehouse/data/OMNI_Report_Warehouse" #For OMNI

]

# List to store the dataframes for final merge
dfs = []

# Iterate over each folder
for folder in folders:
    # List all csv files in the folder
    for file in os.listdir(folder):
        file_path = os.path.join(folder, file)
        # Check if the file is a CSV
        if file.endswith('.csv') and "AstraZe" in file: # Becuase some batches were named as "AstraZenaca" instead of "AstraZeneca"
            try:
                # Determine the header row based on the presence of "MRD" in the filename
                # Some corner cases
                if file in header0_list:
                    df = pd.read_csv(file_path, header=0)
                elif file in header1_list:
                    df = pd.read_csv(file_path, header=1)
                elif "MRD" in file.upper():
                    df = pd.read_csv(file_path, header=0)
                elif "Reveal" in file:
                    df = pd.read_csv(file_path, header=0)
                else:
                    df = pd.read_csv(file_path, header=1)

                # Select specified columns, handling cases where some columns might not exist
                columns = ['Study_ID', 'GHRequestID', 'Customer_SampleID', 'Patient_ID', 'Visit_name', 'Max_percentage', 'Sample_status', 'Sample_comment', 'cfDNA_ng', 'Plasma_ml_input', 'Cancertype']
                df_sub = df.loc[:, df.columns.isin(columns)].copy()

                # Drop duplicates based on 'Customer_SampleID' instead of "GHSampleID" becuase some batches are missing this column
                df_sub_dedup = df_sub.drop_duplicates(subset=["Customer_SampleID"])

                # Add a column for the CSV file name
                df_sub_dedup['csv_file_name'] = file

                # Get the UID of the file owner and convert it to a username
                file_stats = os.stat(file_path)
                file_uid = file_stats.st_uid
                file_creator = pwd.getpwuid(file_uid).pw_name
                df_sub_dedup['file_creator'] = file_creator

                # Append the processed dataframe to the list
                dfs.append(df_sub_dedup)
            except pd.errors.ParserError:
                print(f"Error reading file: {file_path} - possibly incorrect header.")
            except Exception as e:
                print(f"An unexpected error occurred with file: {file_path} - {str(e)}")

# Merge all dataframes into a final DataFrame
df_merged = pd.concat(dfs, ignore_index=True)
df_merged.head()


,Study_ID,Customer_SampleID,GHRequestID,Patient_ID,Visit_name,Sample_status,Sample_comment,Max_percentage,cfDNA_ng,Plasma_ml_input,Cancertype,csv_file_name,file_creator
0,D926NC00001,6219464014,A0890339,E5703003,SCRN,SUCCESS,NaN,NaN,9.620,3.0,NaN,20231127_AstraZeneca_SOW170_1_188_infinityReport.csv,produser
1,D926NC00001,6802134548,A0890340,E4303003,SCRN,SUCCESS,NaN,NaN,119.600,3.5,NaN,20231127_AstraZeneca_SOW170_1_188_infinityReport.csv,produser
2,D926NC00001,6219464013,A0890341,E5703001,SCRN,SUCCESS,NaN,NaN,8.788,3.0,NaN,20231127_AstraZeneca_SOW170_1_188_infinityReport.csv,produser
3,D926NC00001,6220345986,A0890342,E2617001,SCRN,SUCCESS,NaN,NaN,13.000,3.5,NaN,20231127_AstraZeneca_SOW170_1_188_infinityReport.csv,produser
4,D926NC00001,6604395135,A0890343,E6009008,SCRN,SUCCESS,NaN,NaN,7.332,3.5,NaN,20231127_AstraZeneca_SOW170_1_188_infinityReport.csv,produser


### DO NOT RUN! This is for sample checking. START CELL here 

In [2]:
# File name list to handle corner case of data readin
header1_list = [""]
header0_list = [
 '20230928_AstraZeneca_SOW164_46Samples_InfinityReport.csv',
 '20231106_AstraZeneca_SOW164_46Samples_InfinityReport.csv',
 '20231207_AstraZeneca_SOW170_188Samples_InfinityReport.csv',
 '20240111_AstraZeneca_SOW174_35Samples_InfinityReport.csv',
 '20240116_AstraZeneca_SOW182_40Samples_InfinityReport.csv',
 '20240422_AstraZeneca_SOW176Monitoring_998Samples_InfinityReport.csv',

]

# Define the folder paths
folders = [
    # "/ghsfa/projects/pharma/datawarehouse/Infinity", # This only include AZ batches before 2024
    "/ghsfa/projects/pharma/datawarehouse/infinity",
    "/ghsfa/projects/pharma/datawarehouse/data/infinity_ist_report_warehouse",
    "/ghsfa/projects/pharma/datawarehouse/data/infinity_mrd_report_warehouse",
    "/ghsfa/projects/pharma/datawarehouse/data/infinity_report_warehouse",

    "/ghsfa/projects/pharma/datawarehouse/data/g360_reports_since_201902", # For G360
    "/ghsfa/projects/pharma/datawarehouse/data/OMNI_Report_Warehouse" #For OMNI

]

# List to store the dataframes for final merge
dfs = []

# Iterate over each folder
for folder in folders:
    # List all csv files in the folder
    for file in os.listdir(folder):
        file_path = os.path.join(folder, file)
        # Check if the file is a CSV
        if file.endswith('.csv') and "AstraZe" in file: # Becuase some batches were named as "AstraZenaca" instead of "AstraZeneca"
            try:
                # Determine the header row based on the presence of "MRD" in the filename
                # Some corner cases
                if file in header0_list:
                    df = pd.read_csv(file_path, header=0)
                elif file in header1_list:
                    df = pd.read_csv(file_path, header=1)
                elif "MRD" in file.upper():
                    df = pd.read_csv(file_path, header=0)
                elif "Reveal" in file:
                    df = pd.read_csv(file_path, header=0)
                else:
                    df = pd.read_csv(file_path, header=1)

                # Select specified columns, handling cases where some columns might not exist
                columns = ['Study_ID', 'GHRequestID', 'GHSampleID', 'Patient_ID', 'Visit_name', 'Max_percentage', 'Sample_status', 'Sample_comment', 'cfDNA_ng', 'Plasma_ml_input', 'Cancertype']
                df_sub = df.loc[:, df.columns.isin(columns)].copy()

                # Drop duplicates based on 'Customer_SampleID' instead of "GHSampleID" becuase some batches are missing this column
                df_sub_dedup = df_sub.drop_duplicates(subset=["GHSampleID"])

                # Add a column for the CSV file name
                df_sub_dedup['csv_file_name'] = file

                # Get the UID of the file owner and convert it to a username
                file_stats = os.stat(file_path)
                file_uid = file_stats.st_uid
                file_creator = pwd.getpwuid(file_uid).pw_name
                df_sub_dedup['file_creator'] = file_creator

                # Append the processed dataframe to the list
                dfs.append(df_sub_dedup)
            except pd.errors.ParserError:
                print(f"Error reading file: {file_path} - possibly incorrect header.")
            except Exception as e:
                print(f"An unexpected error occurred with file: {file_path} - {str(e)}")

# Merge all dataframes into a final DataFrame
df_merged = pd.concat(dfs, ignore_index=True)
df_merged.head()


An unexpected error occurred with file: /ghsfa/projects/pharma/datawarehouse/data/g360_reports_since_201902/20210912_AstraZeneca_SOW97_6Samples_G360Report.csv - Index(['GHSampleID'], dtype='object')
An unexpected error occurred with file: /ghsfa/projects/pharma/datawarehouse/data/g360_reports_since_201902/20210506_AstraZeneca_SOW90_17Samples_G360Report.csv - Index(['GHSampleID'], dtype='object')
An unexpected error occurred with file: /ghsfa/projects/pharma/datawarehouse/data/g360_reports_since_201902/20210825_AstraZeneca_SOW97_4Samples_G360Report.csv - Index(['GHSampleID'], dtype='object')
An unexpected error occurred with file: /ghsfa/projects/pharma/datawarehouse/data/g360_reports_since_201902/20210809_AstraZeneca_SOW66_28Samples_G360Report.csv - Index(['GHSampleID'], dtype='object')
An unexpected error occurred with file: /ghsfa/projects/pharma/datawarehouse/data/g360_reports_since_201902/20210720_AstraZeneca_SOW100_161Samples_G360Report.csv - Index(['GHSampleID'], dtype='object')


,Study_ID,GHRequestID,GHSampleID,Patient_ID,Visit_name,Sample_status,Sample_comment,Max_percentage,cfDNA_ng,Plasma_ml_input,Cancertype,csv_file_name,file_creator
0,D926NC00001,A0890339,B00535465,E5703003,SCRN,SUCCESS,NaN,NaN,9.620,3.0,NaN,20231127_AstraZeneca_SOW170_1_188_infinityReport.csv,produser
1,D926NC00001,A0890340,B00535427,E4303003,SCRN,SUCCESS,NaN,NaN,119.600,3.5,NaN,20231127_AstraZeneca_SOW170_1_188_infinityReport.csv,produser
2,D926NC00001,A0890341,B00535430,E5703001,SCRN,SUCCESS,NaN,NaN,8.788,3.0,NaN,20231127_AstraZeneca_SOW170_1_188_infinityReport.csv,produser
3,D926NC00001,A0890342,B00535432,E2617001,SCRN,SUCCESS,NaN,NaN,13.000,3.5,NaN,20231127_AstraZeneca_SOW170_1_188_infinityReport.csv,produser
4,D926NC00001,A0890343,B00535434,E6009008,SCRN,SUCCESS,NaN,NaN,7.332,3.5,NaN,20231127_AstraZeneca_SOW170_1_188_infinityReport.csv,produser


In [3]:
# Readin samples 
df_merged_1 = df_merged.copy()

# Drop duplicates 
df_merged_dedup = df_merged_1.drop_duplicates(subset=["GHSampleID"])
print("Total size of this RUO/IST summary data is " + str(df_merged_dedup.shape[0]) + "\n") 

# Regular expression to extract 'Account', 'SOW_num', and 'report_date'
pattern = r'(?P<report_date>^\d+)_?(?P<Account>[^_]+)_([^_]+)'

# Using str.extract() to create new columns in the DataFrame
df_merged_dedup[['report_date', 'Account', 'SOW_num']] = df_merged_dedup['csv_file_name'].str.extract(pattern)

df_merged_dedup.head()

Total size of this RUO/IST summary data is 20361



,Study_ID,GHRequestID,GHSampleID,Patient_ID,Visit_name,Sample_status,Sample_comment,Max_percentage,cfDNA_ng,Plasma_ml_input,Cancertype,csv_file_name,file_creator,report_date,Account,SOW_num
0,D926NC00001,A0890339,B00535465,E5703003,SCRN,SUCCESS,NaN,NaN,9.620,3.0,NaN,20231127_AstraZeneca_SOW170_1_188_infinityReport.csv,produser,20231127,AstraZeneca,SOW170
1,D926NC00001,A0890340,B00535427,E4303003,SCRN,SUCCESS,NaN,NaN,119.600,3.5,NaN,20231127_AstraZeneca_SOW170_1_188_infinityReport.csv,produser,20231127,AstraZeneca,SOW170
2,D926NC00001,A0890341,B00535430,E5703001,SCRN,SUCCESS,NaN,NaN,8.788,3.0,NaN,20231127_AstraZeneca_SOW170_1_188_infinityReport.csv,produser,20231127,AstraZeneca,SOW170
3,D926NC00001,A0890342,B00535432,E2617001,SCRN,SUCCESS,NaN,NaN,13.000,3.5,NaN,20231127_AstraZeneca_SOW170_1_188_infinityReport.csv,produser,20231127,AstraZeneca,SOW170
4,D926NC00001,A0890343,B00535434,E6009008,SCRN,SUCCESS,NaN,NaN,7.332,3.5,NaN,20231127_AstraZeneca_SOW170_1_188_infinityReport.csv,produser,20231127,AstraZeneca,SOW170


In [4]:
# 
# df_merged_dedup_test = df_merged_dedup[df_merged_dedup["GHSampleID"] == "B00310852"]
# df_merged_dedup_test

df_merged_dedup_test = df_merged_dedup[df_merged_dedup["Patient_ID"] == "E4301002"]
df_merged_dedup_test

,Study_ID,GHRequestID,GHSampleID,Patient_ID,Visit_name,Sample_status,Sample_comment,Max_percentage,cfDNA_ng,Plasma_ml_input,Cancertype,csv_file_name,file_creator,report_date,Account,SOW_num
143,D926NC00001,A0890482,B00538694,E4301002,SCRN,SUCCESS,NaN,NaN,14.7160,NaN,NaN,20231127_AstraZeneca_SOW170_1_188_infinityReport.csv,produser,20231127,AstraZeneca,SOW170
381,D933AC00001,A0904208,B00548760,E4301002,C1D1,SUCCESS,NaN,NaN,10.2960,3.0,NaN,20231218_AstraZeneca_SOW157_3_465_infinityReport.csv,produser,20231218,AstraZeneca,SOW157
708,D9970C00001,A1120313,B00742366,E4301002,AESCC2D15,SUCCESS,NaN,NaN,NaN,4.0,NaN,20240909_AstraZeneca_SOW163Monitoring_Batch2_32Samples_MonitoringReport.csv,kchang,20240909,AstraZeneca,SOW163Monitoring
713,D9970C00001,A1120318,B00742371,E4301002,AESCC2D1,SUCCESS,NaN,NaN,NaN,4.0,NaN,20240909_AstraZeneca_SOW163Monitoring_Batch2_32Samples_MonitoringReport.csv,kchang,20240909,AstraZeneca,SOW163Monitoring
719,D9970C00001,A1120300,B00742380,E4301002,AESCC3D1,SUCCESS,NaN,NaN,NaN,4.0,NaN,20240909_AstraZeneca_SOW163Monitoring_Batch2_32Samples_MonitoringReport.csv,kchang,20240909,AstraZeneca,SOW163Monitoring
729,D9970C00001,A1120296,B00742396,E4301002,AESCC1D15,SUCCESS,NaN,NaN,NaN,4.0,NaN,20240909_AstraZeneca_SOW163Monitoring_Batch2_32Samples_MonitoringReport.csv,kchang,20240909,AstraZeneca,SOW163Monitoring
824,D926UC00001,A1088710,B00723793,E4301002,C1D1,SUCCESS,Low Diversity Warning; Unusual Coverage Profile Warning,7.32,2.3200,1.0,NaN,20240812_AstraZeneca_SOW187_Batch3_90Samples_InfinityReport.csv,kchang,20240812,AstraZeneca,SOW187
13308,D9800C00001,A1106012,B00732495,E4301002,S1SCR,SUCCESS,NaN,14.88,64.2400,4.0,NaN,20240819_AstraZeneca_SOW184500genes_Batch1_12Samples_InfinityReport.csv,hazhang,20240819,AstraZeneca,SOW184500genes
13315,D9800C00001,A1106019,B00732491,E4301002,S1EOT,SUCCESS,NaN,37.56,74.1700,4.0,NaN,20240819_AstraZeneca_SOW184500genes_Batch1_12Samples_InfinityReport.csv,hazhang,20240819,AstraZeneca,SOW184500genes
16280,D9311C00001,A0982732,B00624859,E4301002,C3D1,SUCCESS,NaN,0.30,16.1720,3.5,Endometrial Cancer,20240422_AstraZeneca_SOW176_Monitoring_998Samples_InfinityReport.csv,ewarner,20240422,AstraZeneca,SOW176


### DO NOT RUN above! This is for sample checking. START CELL ENDS

In [9]:
# Readin samples 
df_merged_1 = df_merged.copy()

# Drop duplicates 
df_merged_dedup = df_merged_1.drop_duplicates(subset=["Customer_SampleID"])
print("Total size of this RUO/IST summary data is " + str(df_merged_dedup.shape[0]) + "\n") 

# Regular expression to extract 'Account', 'SOW_num', and 'report_date'
pattern = r'(?P<report_date>^\d+)_?(?P<Account>[^_]+)_([^_]+)'

# Using str.extract() to create new columns in the DataFrame
df_merged_dedup[['report_date', 'Account', 'SOW_num']] = df_merged_dedup['csv_file_name'].str.extract(pattern)

df_merged_dedup.head()

Total size of this RUO/IST summary data is 29828



,Study_ID,Customer_SampleID,GHRequestID,Patient_ID,Visit_name,Sample_status,Sample_comment,Max_percentage,cfDNA_ng,Plasma_ml_input,Cancertype,csv_file_name,file_creator,report_date,Account,SOW_num
0,D926NC00001,6219464014,A0890339,E5703003,SCRN,SUCCESS,NaN,NaN,9.620,3.0,NaN,20231127_AstraZeneca_SOW170_1_188_infinityReport.csv,produser,20231127,AstraZeneca,SOW170
1,D926NC00001,6802134548,A0890340,E4303003,SCRN,SUCCESS,NaN,NaN,119.600,3.5,NaN,20231127_AstraZeneca_SOW170_1_188_infinityReport.csv,produser,20231127,AstraZeneca,SOW170
2,D926NC00001,6219464013,A0890341,E5703001,SCRN,SUCCESS,NaN,NaN,8.788,3.0,NaN,20231127_AstraZeneca_SOW170_1_188_infinityReport.csv,produser,20231127,AstraZeneca,SOW170
3,D926NC00001,6220345986,A0890342,E2617001,SCRN,SUCCESS,NaN,NaN,13.000,3.5,NaN,20231127_AstraZeneca_SOW170_1_188_infinityReport.csv,produser,20231127,AstraZeneca,SOW170
4,D926NC00001,6604395135,A0890343,E6009008,SCRN,SUCCESS,NaN,NaN,7.332,3.5,NaN,20231127_AstraZeneca_SOW170_1_188_infinityReport.csv,produser,20231127,AstraZeneca,SOW170


In [5]:
# Subset to 2024 batches of AZ

# Convert 'report_date' to datetime format
df_merged_dedup['report_date'] = pd.to_datetime(df_merged_dedup['report_date'], format='%Y%m%d')

# Filter for entries from the year 2024
df_merged_dedup_2024 = df_merged_dedup[df_merged_dedup['report_date'].dt.year == 2024]

# Continue with your processing...
print("Total size of this RUO/IST summary data for 2024 is " + str(df_merged_dedup_2024.shape[0]) + "\n")

df_merged_dedup_2024.head()


Total size of this RUO/IST summary data for 2024 is 6758



,Study_ID,Customer_SampleID,GHRequestID,Patient_ID,Visit_name,Sample_status,Sample_comment,Max_percentage,cfDNA_ng,Plasma_ml_input,Cancertype,csv_file_name,file_creator,report_date,Account,SOW_num
663,D9720C00003,6604943170,A0994119,E0301019,C1D1B,SUCCESS,NaN,2.63,10.036,3.5,NaN,20240612_AstraZeneca_SOW137_Amnd1_30Samples_500gene_InfinityReport.csv,ewarner,2024-06-12,AstraZeneca,SOW137
664,D9720C00003,6604902860,A0994116,E0302017,C1D1B,SUCCESS,NaN,0.28,8.788,3.5,NaN,20240612_AstraZeneca_SOW137_Amnd1_30Samples_500gene_InfinityReport.csv,ewarner,2024-06-12,AstraZeneca,SOW137
665,D9720C00003,6604902861,A0994120,E0302018,C1D1B,SUCCESS,NaN,0.50,11.544,4.0,NaN,20240612_AstraZeneca_SOW137_Amnd1_30Samples_500gene_InfinityReport.csv,ewarner,2024-06-12,AstraZeneca,SOW137
666,D9720C00003,6604884198,A0994102,E0303008,C1D1B,SUCCESS,NaN,0.98,20.020,4.0,NaN,20240612_AstraZeneca_SOW137_Amnd1_30Samples_500gene_InfinityReport.csv,ewarner,2024-06-12,AstraZeneca,SOW137
667,D9720C00003,6604993819,A0994106,E0303010,C1D1B,SUCCESS,NaN,1.58,11.492,3.5,NaN,20240612_AstraZeneca_SOW137_Amnd1_30Samples_500gene_InfinityReport.csv,ewarner,2024-06-12,AstraZeneca,SOW137


In [6]:
df_merged_dedup_2024.csv_file_name.nunique()

117

In [7]:
df_merged_dedup_2024.Sample_status.value_counts()

SUCCESS    6661
FAIL         97
Name: Sample_status, dtype: int64

In [17]:
df_merged_dedup_2024.SOW_num.nunique()

56

In [11]:
df_merged_dedup_2024.Sample_comment.value_counts()

Possible Contamination Warning                                                       82
Low Diversity Warning                                                                76
Methylation Suppression Warning; Methylation Bias Warning                            55
Failed Enrichment                                                                    27
GC Bias Warning                                                                      24
Low On-Target Rate                                                                   21
Possible Contamination                                                               18
Unusual Coverage Profile Warning                                                     13
Low Diversity Warning; Unusual Coverage Profile Warning                              10
Methylation Bias Warning                                                              9
Low Diversity                                                                         6
Unusual Coverage Profile, Low Di

In [18]:
df_merged_dedup_2024_fail = df_merged_dedup_2024[df_merged_dedup_2024['Sample_status'] == "FAIL"]
print(df_merged_dedup_2024_fail.shape[0])
df_merged_dedup_2024_success = df_merged_dedup_2024[df_merged_dedup_2024['Sample_status'] == "SUCCESS"]
print(df_merged_dedup_2024_success.shape[0])


97
6653


In [20]:
df_merged_dedup_2024_success.Sample_comment.value_counts()

Possible Contamination Warning                                                       82
Low Diversity Warning                                                                74
Methylation Suppression Warning; Methylation Bias Warning                            55
GC Bias Warning                                                                      24
Unusual Coverage Profile Warning                                                     11
Low Diversity Warning; Unusual Coverage Profile Warning                              10
Methylation Bias Warning                                                              9
Low Diversity Warning, Unusal Coverage Profile Warning                                4
Low On Target Rate Warning                                                            3
Low On-Target Rate                                                                    2
Low Diversity Warning; Methylation Suppression Warning                                1
Unusal Coverage Profile Warning 

In [19]:
df_merged_dedup_2024_fail.Sample_comment.value_counts()

Failed Enrichment                                                                   27
Low On-Target Rate                                                                  19
Possible Contamination                                                              18
Low Diversity                                                                        6
Unusual Coverage Profile, Low Diversity                                              4
Low Diversity, Unusual Coverage Profile Warning                                      2
Failed Sequencing                                                                    2
Unusual Coverage Profile Warning                                                     2
Low Diversity Warning                                                                2
Low Diversity Warning; Methylation Suppression Warning                               2
GC Bias                                                                              2
Low Diversity Warning; GC Bias Warning     

In [17]:
# Calculate percentage of FAIL due to Possible Contamination
total_fails = df_merged_dedup_2024[df_merged_dedup_2024['Sample_status'] == 'FAIL'].shape[0]
fails_due_to_contamination = df_merged_dedup_2024[(df_merged_dedup_2024['Sample_status'] == 'FAIL') & (df_merged_dedup_2024['Sample_comment'].str.contains('Possible Contamination', na=False))].shape[0]

if total_fails > 0:
    percentage_fails_due_to_contamination = (fails_due_to_contamination / df_merged_dedup_2024.shape[0]) * 100
else:
    percentage_fails_due_to_contamination = 0

print(f"Percentage of FAIL samples due to Possible Contamination: {percentage_fails_due_to_contamination:.2f}%")

# Calculate percentage of SUCCESS with Possible Contamination warning
total_success = df_merged_dedup_2024[df_merged_dedup_2024['Sample_status'] == 'SUCCESS'].shape[0]
success_with_contamination_warning = df_merged_dedup_2024[(df_merged_dedup_2024['Sample_status'] == 'SUCCESS') & (df_merged_dedup_2024['Sample_comment'].str.contains('Possible Contamination', na=False))].shape[0]

if total_success > 0:
    percentage_success_with_contamination_warning = (success_with_contamination_warning / df_merged_dedup_2024.shape[0]) * 100
else:
    percentage_success_with_contamination_warning = 0

print(f"Percentage of SUCCESS samples with Possible Contamination warning: {percentage_success_with_contamination_warning:.2f}%")


Percentage of FAIL samples due to Possible Contamination: 0.28%
Percentage of SUCCESS samples with Possible Contamination warning: 1.21%


In [ ]:
# Calculate the total sample size for each SOW
total_samples_per_sow = df_merged_dedup_2024.groupby('SOW_num').size().rename('Total Sample Size')

# Calculate FAIL due to Possible Contamination for each SOW
fail_contamination = df_merged_dedup_2024[(df_merged_dedup_2024['Sample_status'] == 'FAIL') &
                                          df_merged_dedup_2024['Sample_comment'].str.contains('Possible Contamination', na=False)]
fail_contamination_count = fail_contamination.groupby('SOW_num').size().rename('Total FAIL due to Possible Contamination')
fail_contamination_percentage = (fail_contamination.groupby('SOW_num').size() / total_samples_per_sow * 100).rename('Percentage of FAIL samples due to Possible Contamination')

# Calculate SUCCESS with Possible Contamination warning for each SOW
success_contamination = df_merged_dedup_2024[(df_merged_dedup_2024['Sample_status'] == 'SUCCESS') &
                                             df_merged_dedup_2024['Sample_comment'].str.contains('Possible Contamination', na=False)]
success_contamination_count = success_contamination.groupby('SOW_num').size().rename('Total SUCCESS with Possible Contamination Warning')
success_contamination_percentage = (success_contamination.groupby('SOW_num').size() / total_samples_per_sow * 100).rename('Percentage of SUCCESS samples with Possible Contamination warning')

# Combine the results into a single DataFrame
sow_contamination_analysis = pd.DataFrame({
    'Total Sample Size': total_samples_per_sow,
    'Total FAIL due to Possible Contamination': fail_contamination_count,
    'Percentage of FAIL due to Possible Contamination': fail_contamination_percentage,
    'Total SUCCESS with Possible Contamination Warning': success_contamination_count,
    'Percentage of SUCCESS with Contamination Warning': success_contamination_percentage
}).fillna(0)  # Filling NaN with 0 where no data is present for either FAIL or SUCCESS

# Display the DataFrame
# print(sow_contamination_analysis)

# Specify the path and save the DataFrame as a CSV file
file_path = f"{work_dir}/sow_contamination_analysis.csv"
sow_contamination_analysis.to_csv(file_path, index=True)
